# Лабораторная работа №5
## Вариант 3
### Выполнил: Федоров А. В. 6211

### Импорты библиотек:

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
from torchvision.transforms import Normalize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from typing import Tuple, Any
from pandas import read_csv
from random import randint
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from numpy import array


### Задание 1. Загружаю исходный датасет:

In [ ]:
data_frame = read_csv('annotation_original.csv')
df = data_frame[["Absolute path", "Class label"]]
df.loc[df['Class label'] == 'cat', 'Class label'] = 0
df.loc[df['Class label'] == 'dog', 'Class label'] = 1
df.to_csv("annotation.csv", index=0)
print(df)

### Класс датасета из лекции:

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, path_to_annotation_file: str, transform: Any = None, target_transform: Any = None) -> None:
        self.path_to_annotation_file = path_to_annotation_file
        self.dataset_info = read_csv(path_to_annotation_file, header=None)
        self.dataset_info.drop(self.dataset_info.columns[[0]], axis=1, inplace=True)
        self.dataset_info.drop(index=0, axis=0, inplace=True)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        return len(self.dataset_info)

    def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
        path_to_image = self.dataset_info.iloc[index, 0]
        image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
        label = self.dataset_info.iloc[index, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_Transform(label)

        return image, label

### Задание 2.

In [ ]:
random_list = list(range(2100))
shuffle(random_list)
train_csv = df.iloc[random_list[:1676]]
train_csv.to_csv("train_csv.csv")
test_csv = df.iloc[random_list[1676:1888]]
test_csv.to_csv("test_csv.csv")
valid_csv = df.iloc[random_list[1888:]]
valid_csv.to_csv("valid_csv.csv")
print(len(train_csv), len(test_csv), len(valid_csv))

### Задание 3. Свёрточная сеть:

In [ ]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()

        self.conv_1 = nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2)
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2)
        self.conv_3 = nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.max_pool = nn.MaxPool2d(2)

        self.fc_1 = nn.Linear(576, 10)
        self.fc_2 = nn.Linear(10, 1)

    def forward(self, x: torch.tensor) -> torch.tensor:
        output = self.relu(self.conv_1(x))
        output = self.max_pool(output)
        output = self.relu(self.conv_2(output))
        output = self.max_pool(output)
        output = self.relu(self.conv_3(output))
        output = self.max_pool(output)

        output = nn.Flatten()(output)
        output = self.relu(self.fc_1(output))
        output = nn.Sigmoid()(self.fc_2(output))
        return output

### Задание 4. Пайплайн предобработки данных + создание выборок.

In [ ]:
custom_transforms = Compose([ToTensor(),
                             Resize((224, 224)),
                             Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                             ])
train_dataset = CustomImageDataset("train_csv.csv", custom_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = CustomImageDataset("test_csv.csv", custom_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)
valid_dataset = CustomImageDataset("valid_csv.csv", custom_transforms)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=False)

### Создание модели. Объявление оптимизатора и критерия.

In [ ]:
device = torch.device("cuda:0")
model1 = CNN().to(device)

optimizer = Adam(params=model1.parameters(), lr=0.001)
criterion = nn.BCELoss()

### Задание 5. Train loop.

In [ ]:
epochs = 20
model1.train()
accuracy_values = []
loss_values = []

for epoch in range(epochs):

    epoch_loss = 0
    epoch_accuracy = 0
    i = 0
    for data, label in train_dataloader:
        i += 1
        print(i)
        data = data.to(device)
        label = torch.tensor(list(int(x) for x in label))
        label = label.to(device)
        output = model1(data)
        loss = criterion(output, label.unsqueeze(dim=1).to(torch.float))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = array(
            ([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(label[j]) else 0 for j in range(4)])).mean()
        epoch_accuracy += acc / len(train_dataloader)
        epoch_loss += loss / len(train_dataloader)

    accuracy_values.append(epoch_accuracy)
    loss_values.append(epoch_loss)
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch + 1, epoch_accuracy, epoch_loss))

### Задание 6. Выводим график функции потерь:

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(range(20), accuracy_values, color="green")
plt.plot(range(20), [float(value.detach()) for value in loss_values], color="blue")
plt.legend(["Accuracy", "Loss"])

### Задание 7. Прогон модели по тестовым данным

In [ ]:
model1.eval()
test_loss = 0
test_accuracy = 0

for data, label in test_dataloader:
    data = data.to(device)
    label = label.to(device)
    
    output = model1(data)
    acc = array(([1 if (1 if output[j][0].detach() >= 0.5 else 0) == int(label[j]) else 0 for j in range(4)])).mean()
    test_accuracy += acc / len(test_dataloader)
    test_loss += float(loss.detach()) / len(test_dataloader)